In [13]:
import pandas as pd
import os

# Path where your parquet parts are stored
dir_path = "/content/drive/MyDrive/datasets/parquet_parts"

# Load first part to inspect
first_part = sorted([f for f in os.listdir(dir_path) if f.endswith('.parquet')])[0]
df = pd.read_parquet(os.path.join(dir_path, first_part))

df.head(), df.info(), df.columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   event_time     200000 non-null  object 
 1   event_type     200000 non-null  object 
 2   product_id     200000 non-null  int64  
 3   category_id    200000 non-null  int64  
 4   category_code  133358 non-null  object 
 5   brand          169680 non-null  object 
 6   price          200000 non-null  float64
 7   user_id        200000 non-null  int64  
 8   user_session   200000 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 13.7+ MB


(                event_time event_type  product_id          category_id  \
 0  2019-11-01 00:00:00 UTC       view     1003461  2053013555631882655   
 1  2019-11-01 00:00:00 UTC       view     5000088  2053013566100866035   
 2  2019-11-01 00:00:01 UTC       view    17302664  2053013553853497655   
 3  2019-11-01 00:00:01 UTC       view     3601530  2053013563810775923   
 4  2019-11-01 00:00:01 UTC       view     1004775  2053013555631882655   
 
                category_code   brand   price    user_id  \
 0     electronics.smartphone  xiaomi  489.07  520088904   
 1  appliances.sewing_machine  janome  293.65  530496790   
 2                       None   creed   28.31  561587266   
 3  appliances.kitchen.washer      lg  712.87  518085591   
 4     electronics.smartphone  xiaomi  183.27  558856683   
 
                            user_session  
 0  4d3b30da-a5e4-49df-b1a8-ba5943f1dd33  
 1  8e5f4f83-366c-4f70-860e-ca7417414283  
 2  755422e7-9040-477b-9bd2-6a6e8fd97387  
 3  3bfb58cd-7

In [14]:

import os, pandas as pd
dir_path = "/content/drive/MyDrive/datasets/parquet_parts"
part_files = sorted([f for f in os.listdir(dir_path) if f.endswith('.parquet')])
print("Found parts:", len(part_files), "example:", part_files[:5])

sample = pd.read_parquet(os.path.join(dir_path, part_files[0]))
print("Sample shape:", sample.shape)
print("\nColumns:\n", sample.columns.tolist())
print("\nEvent type unique values:\n", sample['event_type'].value_counts().to_dict())
print("\nNull counts:\n", sample.isnull().sum())
print("\nSample rows:\n", sample.head().to_dict(orient='records')[:3])


Found parts: 338 example: ['part_000.parquet', 'part_001.parquet', 'part_002.parquet', 'part_003.parquet', 'part_004.parquet']
Sample shape: (200000, 9)

Columns:
 ['event_time', 'event_type', 'product_id', 'category_id', 'category_code', 'brand', 'price', 'user_id', 'user_session']

Event type unique values:
 {'view': 193824, 'purchase': 3462, 'cart': 2714}

Null counts:
 event_time           0
event_type           0
product_id           0
category_id          0
category_code    66642
brand            30320
price                0
user_id              0
user_session         0
dtype: int64

Sample rows:
 [{'event_time': '2019-11-01 00:00:00 UTC', 'event_type': 'view', 'product_id': 1003461, 'category_id': 2053013555631882655, 'category_code': 'electronics.smartphone', 'brand': 'xiaomi', 'price': 489.07, 'user_id': 520088904, 'user_session': '4d3b30da-a5e4-49df-b1a8-ba5943f1dd33'}, {'event_time': '2019-11-01 00:00:00 UTC', 'event_type': 'view', 'product_id': 5000088, 'category_id': 20530

In [15]:
import pandas as pd, os, gc, hashlib, numpy as np, pyarrow as pa, pyarrow.parquet as pq

dir_path = "/content/drive/MyDrive/datasets/parquet_parts"
out_dir = "/content/drive/MyDrive/datasets/processed_sessions_parts"
os.makedirs(out_dir, exist_ok=True)

part_files = sorted([f for f in os.listdir(dir_path) if f.endswith('.parquet')])
print("Processing parts:", len(part_files))

def normalize_event_type(x):
    if pd.isna(x):
        return None
    s = str(x).strip().lower()
    # common mappings — extend if you see other labels
    if s in ('view','pageview','product_view','productview','detailview'): return 'view'
    if s in ('cart','addtocart','add_to_cart','addtocartclick'): return 'cart'
    if s in ('purchase','order','buy','checkout','transaction'): return 'purchase'
    return s

for i, fname in enumerate(part_files):
    path = os.path.join(dir_path, fname)
    df = pd.read_parquet(path)
    # Parse timestamp (remove trailing ' UTC' if present)
    df['event_time'] = df['event_time'].astype(str).str.replace(' UTC','', regex=False)
    df['event_time'] = pd.to_datetime(df['event_time'], errors='coerce', utc=True)
    # Normalize event_type
    df['event_type'] = df['event_type'].apply(normalize_event_type)
    # safe dtypes
    df['user_id'] = df['user_id'].astype('Int64')  # supports missing though we expect none
    # compute per-session aggregates
    grp = df.groupby(['user_id','user_session'], as_index=False).agg(
        first_ts = ('event_time','min'),
        last_ts  = ('event_time','max'),
        events_count = ('event_time','count'),
        unique_products = ('product_id', pd.Series.nunique),
        total_price_sum = ('price','sum'),
        views = (lambda s: (s['event_type']=='view').sum()) if False else ('event_type', lambda s: (s=='view').sum()),
        carts = ('event_type', lambda s: (s=='cart').sum()),
        purchases = ('event_type', lambda s: (s=='purchase').sum())
    )

    grp['session_duration_seconds'] = (grp['last_ts'] - grp['first_ts']).dt.total_seconds().fillna(0).astype(int)
    grp['view_flag'] = (grp['views']>0).astype('int8')
    grp['cart_flag'] = (grp['carts']>0).astype('int8')
    grp['purchase_flag'] = (grp['purchases']>0).astype('int8')

    purchase_rev = df[df['event_type']=='purchase'].groupby(['user_id','user_session'], as_index=False)['price'].sum().rename(columns={'price':'purchase_revenue_part'})
    grp = grp.merge(purchase_rev, on=['user_id','user_session'], how='left')
    grp['purchase_revenue_part'] = grp['purchase_revenue_part'].fillna(0.0)
    # write intermediate file
    out_path = os.path.join(out_dir, f"sessions_part_{i:03d}.parquet")
    grp.to_parquet(out_path, index=False)
    print(f"Wrote {out_path} rows={len(grp)}")
    del df, grp, purchase_rev
    gc.collect()


Processing parts: 338


KeyboardInterrupt: 

In [ ]:

import pandas as pd, os, gc
parts_dir = "/content/drive/MyDrive/datasets/processed_sessions_parts"
parts = sorted([os.path.join(parts_dir,f) for f in os.listdir(parts_dir) if f.endswith('.parquet')])
print("Merging", len(parts), "session-part files")


dfs = []
for p in parts:
    dfp = pd.read_parquet(p)
    dfs.append(dfp)
    print("Loaded", p, "rows:", len(dfp))
all_sessions = pd.concat(dfs, ignore_index=True)
print("Concatenated rows:", len(all_sessions))
del dfs
gc.collect()

# Now groupby to combine session records across parts (a session may appear in multiple raw parts)
agg = all_sessions.groupby(['user_id','user_session'], as_index=False).agg({
    'first_ts': 'min',
    'last_ts': 'max',
    'events_count': 'sum',
    'unique_products': 'sum',   # approximate; rarely duplicated product across parts
    'total_price_sum': 'sum',
    'views': 'sum',
    'carts': 'sum',
    'purchases': 'sum',
    'session_duration_seconds': 'max',
    'view_flag': 'max',
    'cart_flag': 'max',
    'purchase_flag': 'max',
    'purchase_revenue_part': 'sum'
})
# derive final columns
agg['entered_checkout'] = agg['cart_flag']  # define entered_checkout as having at least one cart event
agg['completed_checkout'] = (agg['purchase_flag']>0).astype('int8')
agg['order_value'] = agg['purchase_revenue_part']  # sum of prices for purchases in session
# final tidy cols
final_cols = ['user_id','user_session','first_ts','last_ts','session_duration_seconds',
              'events_count','unique_products','entered_checkout','completed_checkout',
              'order_value','views','carts','purchases']
sessions = agg[final_cols].copy()
print("Final sessions shape:", sessions.shape)
print(sessions.head().T)
# Save final cleaned sessions
out_file = "/content/drive/MyDrive/datasets/cleaned_sessions.parquet"
sessions.to_parquet(out_file, index=False)
print("Wrote final sessions file:", out_file, "size MB:", os.path.getsize(out_file)/1024**2)


In [12]:

import hashlib, numpy as np, pandas as pd, os
sessions = pd.read_parquet("/content/drive/MyDrive/datasets/cleaned_sessions.parquet")
def assign_variant(uid, salt='exp_checkout_v1'):
    h = hashlib.md5(f"{salt}_{int(uid)}".encode()).hexdigest()
    return 'B' if int(h[:8],16) % 2 == 0 else 'A'

sessions['variant'] = sessions['user_id'].apply(assign_variant)
# Basic counts
counts = sessions['variant'].value_counts()
print("Variant counts:\n", counts)
# Conversion by variant
conv = sessions.groupby('variant').agg(
    users = ('user_id','nunique'),
    entered = ('entered_checkout','sum'),
    completed = ('completed_checkout','sum')
).reset_index()
conv['conv_rate_pct'] = 100 * conv['completed'] / conv['entered'].replace(0, pd.NA)
print("\nConversion summary:\n", conv)
# Balance check for events_count
print("\nEvents_count mean by variant:")
print(sessions.groupby('variant')['events_count'].agg(['count','mean','std']))
# Save sessions with variant
sessions.to_parquet("/content/drive/MyDrive/datasets/cleaned_sessions_with_variant.parquet", index=False)
print("Saved sessions with variant.")


Variant counts:
 variant
A    6890887
B    6885754
Name: count, dtype: int64

Conversion summary:
   variant    users  entered  completed  conv_rate_pct
0       A  1849192   873496     388304       44.45401
1       B  1846925   869848     384910       44.25026

Events_count mean by variant:
           count      mean       std
variant                             
A        6890887  4.899760  7.728027
B        6885754  4.899722  8.022859
Saved sessions with variant.


In [16]:
import pandas as pd

sessions = pd.read_parquet("/content/drive/MyDrive/datasets/cleaned_sessions_with_variant.parquet")

overall_entered = sessions['entered_checkout'].sum()
overall_completed = sessions['completed_checkout'].sum()

baseline_conv = overall_completed / overall_entered

baseline_conv


np.float64(0.4435234813094834)

In [17]:
import numpy as np
import pandas as pd

sessions = pd.read_parquet("/content/drive/MyDrive/datasets/cleaned_sessions_with_variant.parquet")

# Extract baseline conversion (actual)
baseline = (sessions['completed_checkout'].sum() /
            sessions['entered_checkout'].sum())

print("Baseline conversion:", baseline)


absolute_uplift = 0.02  # +2 percentage point uplift

# Target conversion for B
target_conv_B = baseline + absolute_uplift
print("Target conversion for B:", target_conv_B)


sessions['completed_sim'] = sessions['completed_checkout']

# For variant B → simulate improved conversions
mask = (sessions['variant'] == 'B') & (sessions['entered_checkout'] == 1)

sessions.loc[mask, 'completed_sim'] = np.random.binomial(
    n=1,
    p=target_conv_B,
    size=mask.sum()
)

# Save updated dataset
sessions.to_parquet("/content/drive/MyDrive/datasets/sessions_with_simulated_uplift.parquet", index=False)

print("Simulation complete. New file saved.")


Baseline conversion: 0.4435234813094834
Target conversion for B: 0.4635234813094834


/tmp/ipython-input-2743145780.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0 0 0 ... 1 0 1]' has dtype incompatible with int8, please explicitly cast to a compatible dtype first.
  sessions.loc[mask, 'completed_sim'] = np.random.binomial(


Simulation complete. New file saved.


In [18]:
sim = pd.read_parquet("/content/drive/MyDrive/datasets/sessions_with_simulated_uplift.parquet")

summary = sim.groupby('variant').agg(
    users=('user_id','nunique'),
    entered=('entered_checkout','sum'),
    completed=('completed_sim','sum')
)

summary['conversion_pct'] = 100 * summary['completed'] / summary['entered']
summary


,users,entered,completed,conversion_pct
variant,,,,
A,1849192,873496,388304,44.454010
B,1846925,869848,465544,53.520155


In [19]:
import pandas as pd

sessions = pd.read_parquet("/content/drive/MyDrive/datasets/sessions_with_simulated_uplift.parquet")

summary = sessions.groupby('variant').agg(
    entered=('entered_checkout','sum'),
    completed=('completed_sim','sum')
).reset_index()

summary


,variant,entered,completed
0,A,873496,388304
1,B,869848,465544


In [20]:
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

count = summary['completed'].values
nobs = summary['entered'].values

z_stat, p_val = proportions_ztest(count, nobs, alternative='two-sided')

print("Z-statistic:", z_stat)
print("p-value:", p_val)


Z-statistic: -119.73030150619034
p-value: 0.0


In [21]:
from statsmodels.stats.proportion import confint_proportions_2indep

conv_A = summary.loc[summary['variant']=='A','completed'].values[0] / summary.loc[summary['variant']=='A','entered'].values[0]
conv_B = summary.loc[summary['variant']=='B','completed'].values[0] / summary.loc[summary['variant']=='B','entered'].values[0]

ci_low, ci_high = confint_proportions_2indep(
    count1=summary.loc[summary['variant']=='B','completed'].values[0],
    nobs1=summary.loc[summary['variant']=='B','entered'].values[0],
    count2=summary.loc[summary['variant']=='A','completed'].values[0],
    nobs2=summary.loc[summary['variant']=='A','entered'].values[0],
    method='wald'
)

print("Conversion A:", conv_A)
print("Conversion B:", conv_B)
print("Uplift:", conv_B - conv_A)
print("95% CI for uplift:", (ci_low, ci_high))


Conversion A: 0.44454010092776614
Conversion B: 0.5352015524551416
Uplift: 0.09066145152737543
95% CI for uplift: (np.float64(0.0891834424951851), np.float64(0.09213946055956575))


In [22]:
import numpy as np

def cohens_h(p1, p2):
    return 2 * (np.arcsin(np.sqrt(p1)) - np.arcsin(np.sqrt(p2)))

h = cohens_h(conv_B, conv_A)
h


np.float64(0.18160990644226627)

In [23]:
import pandas as pd

sessions = pd.read_parquet("/content/drive/MyDrive/datasets/sessions_with_simulated_uplift.parquet")

import os

dir_path = "/content/drive/MyDrive/datasets/parquet_parts"
parts = sorted([f for f in os.listdir(dir_path) if f.endswith('.parquet')])

# Only load necessary columns for mapping
cat_map_list = []
for p in parts[:20]:
    dfp = pd.read_parquet(os.path.join(dir_path,p), columns=['user_id','user_session','category_code'])
    cat_map_list.append(dfp)

cat_df = pd.concat(cat_map_list, ignore_index=True)
cat_df['parent_category'] = cat_df['category_code'].str.split('.', expand=True)[0]


session_category = (
    cat_df
    .groupby(['user_id','user_session'])['parent_category']
    .agg(lambda x: x.value_counts().idxmax() if x.notna().sum()>0 else None)
    .reset_index()
)

# Merge into sessions
sessions = sessions.merge(session_category, on=['user_id','user_session'], how='left')

sessions.to_parquet("/content/drive/MyDrive/datasets/sessions_with_segments.parquet", index=False)

sessions.head()


,user_id,user_session,first_ts,last_ts,session_duration_seconds,events_count,unique_products,entered_checkout,completed_checkout,order_value,views,carts,purchases,variant,completed_sim,parent_category
0,10300217,d1fdcbf1-bb1f-434b-8f1a-4b77f29a84a0,2019-11-06 06:51:52+00:00,2019-11-06 06:51:52+00:00,0,1,1,0,0,0.0,1,0,0,B,0,NaN
1,29515875,1552ef43-4001-4ab7-a9c7-44e99f1730a9,2019-11-12 03:42:01+00:00,2019-11-12 03:42:06+00:00,5,2,1,0,0,0.0,2,0,0,A,0,NaN
2,29515875,27452feb-1b63-41cc-ac67-030d56f6d448,2019-11-12 06:03:02+00:00,2019-11-12 06:04:48+00:00,106,4,2,0,0,0.0,4,0,0,A,0,NaN
3,29515875,447d6309-4efe-4662-a64d-b19992fbfe1c,2019-11-12 05:37:30+00:00,2019-11-12 05:37:30+00:00,0,1,1,0,0,0.0,1,0,0,A,0,NaN
4,29515875,5dd49822-9349-42d6-a165-cd4642ee606b,2019-11-10 02:08:39+00:00,2019-11-10 02:08:39+00:00,0,1,1,0,0,0.0,1,0,0,A,0,NaN


In [4]:
sessions = pd.read_parquet("/content/drive/MyDrive/datasets/sessions_with_segments.parquet")

seg = (
    sessions.groupby(['parent_category','variant'])
    .agg(entered=('entered_checkout','sum'),
         completed=('completed_sim','sum'))
    .reset_index()
)

# Compute conversion and pivot
seg['conversion'] = seg['completed'] / seg['entered'].replace(0, pd.NA)
pivot = seg.pivot(index='parent_category', columns='variant', values='conversion')

pivot['uplift'] = pivot['B'] - pivot['A']
pivot


variant,A,B,uplift
parent_category,,,
accessories,11.285714,6.333333,-4.952381
apparel,9.944444,21.933333,11.988889
appliances,3.286426,3.16624,-0.120185
auto,1.663102,1.789157,0.126055
computers,2.949153,3.427046,0.477894
construction,1.805755,1.626582,-0.179173
country_yard,<NA>,<NA>,<NA>
electronics,1.102651,0.981758,-0.120893
furniture,11.826087,16.16129,4.335203


In [3]:
import pandas as pd

sessions = pd.read_parquet("/content/drive/MyDrive/datasets/sessions_with_segments.parquet")

# Define price bands
sessions['price_band'] = pd.cut(
    sessions['order_value'],
    bins=[-0.1, 50, 200, 500, 2000, 10000],
    labels=['Very Low', 'Low', 'Mid', 'High', 'Premium']
)

seg_price = (
    sessions.groupby(['price_band','variant'], observed=False)
    .agg(entered=('entered_checkout','sum'),
         completed=('completed_sim','sum'))
    .reset_index()
)

seg_price['conversion'] = seg_price['completed'] / seg_price['entered'].replace(0, pd.NA)
pivot_price = seg_price.pivot(index='price_band', columns='variant', values='conversion')
pivot_price['uplift'] = pivot_price['B'] - pivot_price['A']

pivot_price

variant,A,B,uplift
price_band,,,
Very Low,0.101757,0.485554,0.383796
Low,1.196808,0.662798,-0.534011
Mid,1.162850,0.630502,-0.532348
High,1.168830,0.627408,-0.541422
Premium,1.095518,0.551616,-0.543902


In [5]:
sessions['duration_bucket'] = pd.cut(
    sessions['session_duration_seconds'],
    bins=[0, 30, 120, 300, 1200, 100000],
    labels=['0–30s','30–120s','2–5 mins','5–20 mins','20+ mins']
)

seg_dur = (
    sessions.groupby(['duration_bucket','variant'])
    .agg(entered=('entered_checkout','sum'),
         completed=('completed_sim','sum'))
    .reset_index()
)

seg_dur['conversion'] = seg_dur['completed'] / seg_dur['entered'].replace(0,pd.NA)
pivot_dur = seg_dur.pivot(index='duration_bucket', columns='variant', values='conversion')
pivot_dur['uplift'] = pivot_dur['B'] - pivot_dur['A']

pivot_dur


/tmp/ipython-input-3404089664.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sessions.groupby(['duration_bucket','variant'])


variant,A,B,uplift
duration_bucket,,,
0–30s,0.099734,0.479558,0.379824
30–120s,0.445505,0.535420,0.089915
2–5 mins,0.554815,0.549746,-0.005069
5–20 mins,0.475096,0.537186,0.062090
20+ mins,0.367648,0.520387,0.152739


In [6]:

import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import norm
import matplotlib.pyplot as plt

DATA_PATH = "/content/drive/MyDrive/datasets/sessions_with_segments.parquet"  # update if needed

print("Loading:", DATA_PATH)
df = pd.read_parquet(DATA_PATH)
print("Rows:", len(df), "Unique users:", df['user_id'].nunique())
# ensure datetimes and dtypes
df['first_ts'] = pd.to_datetime(df['first_ts'], utc=True)
for c in ['entered_checkout','completed_sim','views','carts','purchases']:
    if c in df.columns:
        df[c] = df[c].fillna(0).astype(int)
# ensure price_band/duration_bucket exist
if 'price_band' not in df.columns:
    df['price_band'] = pd.cut(df['order_value'].fillna(0),
                              bins=[-0.1,50,200,500,2000,1e9],
                              labels=['Very Low','Low','Mid','High','Premium'])
if 'duration_bucket' not in df.columns:
    df['duration_bucket'] = pd.cut(df['session_duration_seconds'].fillna(0),
                                   bins=[0,30,120,300,1200,1e9],
                                   labels=['0–30s','30–120s','2–5 mins','5–20 mins','20+ mins'])
print("Loaded and prepared dataset.")


Loading: /content/drive/MyDrive/datasets/sessions_with_segments.parquet
Rows: 13776641 Unique users: 3696117
Loaded and prepared dataset.


In [7]:

total_sessions = len(df)
total_users = df['user_id'].nunique()
variant_counts = df['variant'].value_counts().reindex(['A','B']).fillna(0).astype(int)

entered_by_var = df.groupby('variant')['entered_checkout'].sum().reindex(['A','B']).fillna(0).astype(int)
completed_by_var = df.groupby('variant')['completed_sim'].sum().reindex(['A','B']).fillna(0).astype(int)

convA = completed_by_var['A'] / entered_by_var['A']
convB = completed_by_var['B'] / entered_by_var['B']
uplift = convB - convA
rel_uplift = (convB/convA - 1) * 100

print(f"Total sessions: {total_sessions:,}")
print(f"Total users: {total_users:,}")
print("Variant counts (sessions):")
print(variant_counts.to_string())
print("\nEntered_checkout by variant:")
print(entered_by_var.to_string())
print("\nCompleted (sim) by variant:")
print(completed_by_var.to_string())
print(f"\nConversion A: {convA:.4f} ({convA*100:.2f}%)")
print(f"Conversion B: {convB:.4f} ({convB*100:.2f}%)")
print(f"Absolute uplift (pp): {uplift*100:.2f} pp")
print(f"Relative uplift: {rel_uplift:.2f}%")


Total sessions: 13,776,641
Total users: 3,696,117
Variant counts (sessions):
variant
A    6890887
B    6885754

Entered_checkout by variant:
variant
A    873496
B    869848

Completed (sim) by variant:
variant
A    388304
B    465544

Conversion A: 0.4445 (44.45%)
Conversion B: 0.5352 (53.52%)
Absolute uplift (pp): 9.07 pp
Relative uplift: 20.39%


In [8]:

count = np.array([int(completed_by_var['A']), int(completed_by_var['B'])])
nobs = np.array([int(entered_by_var['A']), int(entered_by_var['B'])])
# proportions_ztest expects counts and nobs, order doesn't matter; we will test B vs A by providing [B,A]
z_stat, p_val = proportions_ztest([int(completed_by_var['B']), int(completed_by_var['A'])],
                                  [int(entered_by_var['B']), int(entered_by_var['A'])],
                                  alternative='two-sided')
# compute wald CI for difference pB - pA
pA = convA; pB = convB
se = np.sqrt(pA*(1-pA)/nobs[0] + pB*(1-pB)/nobs[1])
z = norm.ppf(0.975)
ci_low = (pB - pA) - z*se
ci_high = (pB - pA) + z*se

print("Z-statistic:", z_stat)
print("p-value:", p_val)
print(f"95% CI for uplift (B - A): [{ci_low:.4f}, {ci_high:.4f}]  => [{ci_low*100:.2f} pp, {ci_high*100:.2f} pp]")


Z-statistic: 119.73030150619034
p-value: 0.0
95% CI for uplift (B - A): [0.0892, 0.0921]  => [8.92 pp, 9.21 pp]


In [9]:

variants = ['A','B']
convs = [convA*100, convB*100]
entries = [int(entered_by_var['A']), int(entered_by_var['B'])]
comps = [int(completed_by_var['A']), int(completed_by_var['B'])]

# compute CIs per variant
ci_lows = []
ci_highs = []
for comp, n in zip(comps, entries):
    p = comp / n if n>0 else 0
    se_p = np.sqrt(p*(1-p)/n) if n>0 else 0
    ci_lows.append(max(0,(p - 1.96*se_p)*100))
    ci_highs.append(min(100,(p + 1.96*se_p)*100))

fig = go.Figure()
fig.add_trace(go.Bar(x=variants, y=convs, text=[f"{c:.2f}%" for c in convs], textposition='auto'))
# CI bands
for i,(low,high) in enumerate(zip(ci_lows, ci_highs)):
    fig.add_shape(type='rect', x0=i-0.15, x1=i+0.15, y0=low, y1=high, fillcolor='rgba(0,0,0,0.06)', line_width=0)
    fig.add_shape(type='line', x0=i-0.2, x1=i+0.2, y0=low, y1=low, line=dict(color='black', width=2))
    fig.add_shape(type='line', x0=i-0.2, x1=i+0.2, y0=high, y1=high, line=dict(color='black', width=2))
fig.update_layout(title="Conversion Rate by Variant (with 95% CI)", yaxis_title="Conversion %", xaxis_title="Variant", yaxis=dict(range=[0, max(convs)*1.2]))
fig.show()


In [10]:

funnel = []
for v in ['A','B']:
    sub = df[df['variant']==v]
    views = int(sub['views'].sum()) if 'views' in sub.columns else 0
    carts = int(sub['carts'].sum()) if 'carts' in sub.columns else 0
    purchases = int(sub['purchases'].sum()) if 'purchases' in sub.columns else 0
    funnel.append({'variant':v,'view':views,'cart':carts,'purchase':purchases})

funnel_df = pd.DataFrame(funnel).set_index('variant')
print(funnel_df)
# Plot grouped bars
funnel_long = funnel_df.reset_index().melt(id_vars='variant', value_vars=['view','cart','purchase'], var_name='stage', value_name='count')
fig = px.bar(funnel_long, x='stage', y='count', color='variant', barmode='group', title='Funnel counts by Variant')
fig.show()


             view     cart  purchase
variant                             
A        31788232  1515082    460378
B        31767878  1513838    456561


In [11]:

tmp = df[df['entered_checkout']==1].copy()
tmp['date'] = tmp['first_ts'].dt.floor('D')
agg = tmp.groupby(['date','variant']).agg(entered=('entered_checkout','sum'), completed=('completed_sim','sum')).reset_index()
agg = agg.sort_values(['date','variant'])
agg['cum_entered'] = agg.groupby('variant')['entered'].cumsum()
agg['cum_completed'] = agg.groupby('variant')['completed'].cumsum()
agg['cum_conv'] = agg['cum_completed'] / agg['cum_entered']

fig = px.line(agg, x='date', y='cum_conv', color='variant', title='Cumulative conversion over time (daily)')
fig.update_yaxes(tickformat=".2%")
fig.show()


In [12]:

cat = df.groupby(['parent_category','variant']).agg(entered=('entered_checkout','sum'), completed=('completed_sim','sum')).reset_index()
# compute conversions
rows = []
for cat_name in sorted(cat['parent_category'].dropna().unique()):
    subset = cat[cat['parent_category']==cat_name]
    a = subset[subset['variant']=='A']
    b = subset[subset['variant']=='B']
    a_enter = int(a['entered'].sum()) if not a.empty else 0
    a_comp = int(a['completed'].sum()) if not a.empty else 0
    b_enter = int(b['entered'].sum()) if not b.empty else 0
    b_comp = int(b['completed'].sum()) if not b.empty else 0
    convA = a_comp/a_enter if a_enter>0 else np.nan
    convB = b_comp/b_enter if b_enter>0 else np.nan
    uplift_val = convB - convA if (not np.isnan(convA) and not np.isnan(convB)) else np.nan
    rows.append({'category':cat_name, 'convA':convA, 'convB':convB, 'uplift':uplift_val, 'A_enter':a_enter, 'B_enter':b_enter})
cat_df = pd.DataFrame(rows).sort_values('uplift', ascending=False)
display(cat_df.head(20))
# bar chart top 20
fig = px.bar(cat_df.head(20), x='category', y='uplift', title='Top 20 Parent Category Uplifts (B - A)')
fig.update_xaxes(tickangle=45)
fig.show()


,category,convA,convB,uplift,A_enter,B_enter
1,apparel,9.944444,21.933333,11.988889,36,15
11,sport,5.750000,13.500000,7.750000,4,2
8,furniture,11.826087,16.161290,4.335203,46,31
12,stationery,2.000000,2.666667,0.666667,2,3
4,computers,2.949153,3.427046,0.477894,295,281
9,kids,5.152174,5.333333,0.181159,46,42
3,auto,1.663102,1.789157,0.126055,187,166
10,medicine,1.166667,1.181818,0.015152,6,11
2,appliances,3.286426,3.166240,-0.120185,803,782
7,electronics,1.102651,0.981758,-0.120893,13502,13321


In [13]:

price = df.groupby(['price_band','variant']).agg(entered=('entered_checkout','sum'), completed=('completed_sim','sum')).reset_index()
price_rows = []
for b in price['price_band'].cat.categories:
    sub = price[price['price_band']==b]
    a_enter = int(sub[(sub['variant']=='A')]['entered'].sum()) if not sub.empty else 0
    a_comp = int(sub[(sub['variant']=='A')]['completed'].sum()) if not sub.empty else 0
    b_enter = int(sub[(sub['variant']=='B')]['entered'].sum()) if not sub.empty else 0
    b_comp = int(sub[(sub['variant']=='B')]['completed'].sum()) if not sub.empty else 0
    convA = a_comp/a_enter if a_enter>0 else np.nan
    convB = b_comp/b_enter if b_enter>0 else np.nan
    price_rows.append({'price_band':b, 'convA':convA, 'convB':convB, 'uplift':convB-convA})
price_df = pd.DataFrame(price_rows)
print(price_df)
fig = px.bar(price_df, x='price_band', y='uplift', title='Uplift by Price Band (B - A)')
fig.show()

# duration uplift
dur = df.groupby(['duration_bucket','variant']).agg(entered=('entered_checkout','sum'), completed=('completed_sim','sum')).reset_index()
dur_rows=[]
for b in dur['duration_bucket'].cat.categories:
    sub = dur[dur['duration_bucket']==b]
    a_enter = int(sub[(sub['variant']=='A')]['entered'].sum()) if not sub.empty else 0
    a_comp = int(sub[(sub['variant']=='A')]['completed'].sum()) if not sub.empty else 0
    b_enter = int(sub[(sub['variant']=='B')]['entered'].sum()) if not sub.empty else 0
    b_comp = int(sub[(sub['variant']=='B')]['completed'].sum()) if not sub.empty else 0
    convA = a_comp/a_enter if a_enter>0 else np.nan
    convB = b_comp/b_enter if b_enter>0 else np.nan
    dur_rows.append({'duration_bucket':b, 'convA':convA, 'convB':convB, 'uplift':convB-convA})
dur_df = pd.DataFrame(dur_rows)
print(dur_df)
fig = px.bar(dur_df, x='duration_bucket', y='uplift', title='Uplift by Session Duration (B - A)')
fig.show()


/tmp/ipython-input-3033431073.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



  price_band     convA     convB    uplift
0   Very Low  0.101757  0.485554  0.383796
1        Low  1.196808  0.662798 -0.534011
2        Mid  1.162850  0.630502 -0.532348
3       High  1.168830  0.627408 -0.541422
4    Premium  1.095250  0.551377 -0.543872


/tmp/ipython-input-3033431073.py:18: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



  duration_bucket     convA     convB    uplift
0           0–30s  0.099734  0.479558  0.379824
1         30–120s  0.445505  0.535420  0.089915
2        2–5 mins  0.554815  0.549746 -0.005069
3       5–20 mins  0.475096  0.537186  0.062090
4        20+ mins  0.367648  0.520387  0.152739


In [1]:
# FIX — Ensure price_band and duration_bucket exist
import pandas as pd
import numpy as np

DATA_PATH = "/content/drive/MyDrive/datasets/sessions_with_segments.parquet"
df = pd.read_parquet(DATA_PATH)

# Convert first_ts if needed
df['first_ts'] = pd.to_datetime(df['first_ts'], utc=True)

# 1. Add price_band if missing
if 'price_band' not in df.columns:
    print("price_band missing — creating it now...")
    df['price_band'] = pd.cut(
        df['order_value'].fillna(0),
        bins=[-0.1, 50, 200, 500, 2000, 1e12],
        labels=['Very Low', 'Low', 'Mid', 'High', 'Premium']
    )

# 2. Add duration_bucket if missing
if 'duration_bucket' not in df.columns:
    print("duration_bucket missing — creating it now...")
    df['duration_bucket'] = pd.cut(
        df['session_duration_seconds'].fillna(0),
        bins=[0, 30, 120, 300, 1200, 1e12],
        labels=['0–30s', '30–120s', '2–5 mins', '5–20 mins', '20+ mins']
    )

# 3. Save updated file
OUTPUT_PATH = "/content/drive/MyDrive/datasets/sessions_with_segments_fixed.parquet"
df.to_parquet(OUTPUT_PATH, index=False)

print("Updated file saved at:", OUTPUT_PATH)
print("Columns now include:", df.columns.tolist())


price_band missing — creating it now...
duration_bucket missing — creating it now...
Updated file saved at: /content/drive/MyDrive/datasets/sessions_with_segments_fixed.parquet
Columns now include: ['user_id', 'user_session', 'first_ts', 'last_ts', 'session_duration_seconds', 'events_count', 'unique_products', 'entered_checkout', 'completed_checkout', 'order_value', 'views', 'carts', 'purchases', 'variant', 'completed_sim', 'parent_category', 'price_band', 'duration_bucket']


In [2]:
import numpy as np
import pandas as pd
from statsmodels.stats.power import NormalIndPower
from math import asin, sqrt, ceil

# --- Config: update path if needed
DATA_PATH = "/content/drive/MyDrive/datasets/sessions_with_segments_fixed.parquet"

# Load data (light)
df = pd.read_parquet(DATA_PATH)
print("Loaded rows:", len(df))

# Baseline conversion (use entered_checkout as unit)
total_entered = df['entered_checkout'].sum()
total_completed = df['completed_sim'].sum()
baseline = total_completed / total_entered
print(f"Observed baseline conversion (overall entered → completed): {baseline:.6f} ({baseline*100:.2f}%)")

# --- Inputs you can edit:
alpha = 0.05         # significance level
power = 0.8          # desired power
mde_absolute = 0.02  # absolute minimum detectable effect (in proportion points, e.g. 0.02 => +2 pp)

# target conversion
p0 = baseline
p1 = baseline + mde_absolute
if p1 >= 1.0:
    raise ValueError("Requested p1 >= 1.0; choose smaller MDE")

# --- compute Cohen's h (effect size for proportions)
def cohens_h(p1, p0):
    return 2 * (asin(sqrt(p1)) - asin(sqrt(p0)))

effect_size = cohens_h(p1, p0)
print(f"Effect size (Cohen's h) for p0={p0:.4f} -> p1={p1:.4f}: h = {effect_size:.6f}")

# --- solve for sample size per arm (in ENTERED units)
solver = NormalIndPower()
n_per_arm = solver.solve_power(effect_size=effect_size, power=power, alpha=alpha, alternative='two-sided')
n_per_arm = int(ceil(n_per_arm))
print(f"Required sample (per arm) of ENTERED_CHECKOUT units: {n_per_arm:,}")

# Compare with current data (entered per variant)
entered_by_variant = df.groupby('variant')['entered_checkout'].sum().reindex(['A','B']).fillna(0).astype(int)
print("\nCurrent entered-checkout by variant:")
print(entered_by_variant.to_string())

# Show whether current data meets required sample per arm
current_A = int(entered_by_variant.loc['A'])
current_B = int(entered_by_variant.loc['B'])
print("\nPower check:")
print(f" - Current A entered: {current_A:,}  (meets requirement? {'YES' if current_A >= n_per_arm else 'NO'})")
print(f" - Current B entered: {current_B:,}  (meets requirement? {'YES' if current_B >= n_per_arm else 'NO'})")

# If you prefer sample size in USERS rather than entered-checkout events:
entered_per_user = df.groupby('user_id')['entered_checkout'].sum().mean()
approx_users_per_arm = int(ceil(n_per_arm / entered_per_user)) if entered_per_user>0 else None
print(f"\nAverage entered-checkout events per user (observed): {entered_per_user:.4f}")
if approx_users_per_arm:
    print(f"Approximate required DISTINCT USERS per arm (estimate): {approx_users_per_arm:,}")
else:
    print("Approximate required DISTINCT USERS per arm (estimate): N/A")


Loaded rows: 13776641
Observed baseline conversion (overall entered → completed): 0.489776 (48.98%)
Effect size (Cohen's h) for p0=0.4898 -> p1=0.5098: h = 0.040003
Required sample (per arm) of ENTERED_CHECKOUT units: 9,810

Current entered-checkout by variant:
variant
A    873496
B    869848

Power check:
 - Current A entered: 873,496  (meets requirement? YES)
 - Current B entered: 869,848  (meets requirement? YES)

Average entered-checkout events per user (observed): 0.4717
Approximate required DISTINCT USERS per arm (estimate): 20,799


In [3]:
import pandas as pd
import numpy as np

DATA_PATH = "/content/drive/MyDrive/datasets/sessions_with_segments_fixed.parquet"
df = pd.read_parquet(DATA_PATH)
df['first_ts'] = pd.to_datetime(df['first_ts'], utc=True)

# Basic guardrail metrics by variant
guard = df.groupby('variant').agg(
    users=('user_id','nunique'),
    sessions=('user_session','nunique'),
    entered=('entered_checkout','sum'),
    completed=('completed_sim','sum'),
    total_revenue=('order_value','sum')
).reset_index()

# Derived metrics
guard['conv_pct'] = guard['completed'] / guard['entered']
guard['rev_per_session'] = guard['total_revenue'] / guard['sessions']
guard['rev_per_user'] = guard['total_revenue'] / guard['users']

print("Guardrail summary per variant:")
display(guard)

# Bounce rate: sessions with exactly 1 event_count (approximation)
sess_events = df.groupby('user_session').agg(events=('events_count','sum'), variant=('variant','first')).reset_index()
bounce_sessions_df = sess_events.loc[sess_events['events'] == 1].groupby('variant').agg(bounce_count=('user_session','count')).reset_index()

# Merge bounce counts into guard to compute bounce rate (%) safely
guard = guard.merge(bounce_sessions_df, on='variant', how='left')
guard['bounce_count'] = guard['bounce_count'].fillna(0).astype(int)
guard['bounce_rate_pct'] = guard['bounce_count'] / guard['sessions'] * 100

print("\nGuardrail with bounce rate (approx):")
display(guard[['variant','users','sessions','entered','completed','total_revenue','conv_pct','rev_per_session','rev_per_user','bounce_count','bounce_rate_pct']])

# Additional useful check: sessions per user distribution summary (to spot anomalies)
spu = df.groupby('user_id')['user_session'].nunique()
print("\nSessions per user — summary (count of sessions per user):")
display(spu.describe().to_frame(name='value'))

# If you want, print any large deviations of bounce rate by variant:
print("\nBounce rate by variant (rounded):")
print(guard.set_index('variant')[['bounce_rate_pct']].round(3).to_string())


Guardrail summary per variant:


,variant,users,sessions,entered,completed,total_revenue,conv_pct,rev_per_session,rev_per_user
0,A,1849192,6890745,873496,388304,137996757.5,0.444540,20.026392,74.625435
1,B,1846925,6885609,869848,465544,137198133.0,0.535202,19.925345,74.284626



Guardrail with bounce rate (approx):


,variant,users,sessions,entered,completed,total_revenue,conv_pct,rev_per_session,rev_per_user,bounce_count,bounce_rate_pct
0,A,1849192,6890745,873496,388304,137996757.5,0.444540,20.026392,74.625435,2495915,36.221265
1,B,1846925,6885609,869848,465544,137198133.0,0.535202,19.925345,74.284626,2505052,36.380980



Sessions per user — summary (count of sessions per user):


,value
count,3.696117e+06
mean,3.727328e+00
std,1.704187e+01
min,1.000000e+00
25%,1.000000e+00
50%,2.000000e+00
75%,4.000000e+00
max,2.254200e+04



Bounce rate by variant (rounded):
         bounce_rate_pct
variant                 
A                 36.221
B                 36.381


In [4]:
# mark returning if user appears more than once in dataset
user_session_counts = df.groupby('user_id')['user_session'].nunique().reset_index().rename(columns={'user_session':'n_sessions'})
user_session_counts['user_type'] = np.where(user_session_counts['n_sessions']>1, 'returning', 'new')
df = df.merge(user_session_counts[['user_id','user_type']], on='user_id', how='left')

cohort = df.groupby(['user_type','variant']).agg(
    entered=('entered_checkout','sum'),
    completed=('completed_sim','sum'),
    users=('user_id','nunique')
).reset_index()
cohort['conv'] = cohort['completed'] / cohort['entered']
cohort_pivot = cohort.pivot(index='user_type', columns='variant', values=['entered','completed','conv'])
display(cohort)
print("\nInterpretation: check conversion differences between new & returning users.")


,user_type,variant,entered,completed,users,conv
0,new,A,60503,30693,820658,0.507297
1,new,B,60057,32959,818514,0.548795
2,returning,A,812993,357611,1028534,0.439870
3,returning,B,809791,432585,1028411,0.534193



Interpretation: check conversion differences between new & returning users.


In [5]:
# we estimate incremental conversions and revenue per entered session
agg = df.groupby('variant').agg(entered=('entered_checkout','sum'), completed=('completed_sim','sum'), revenue=('order_value','sum')).reset_index()
agg['conv'] = agg['completed']/agg['entered']
agg['rev_per_entered'] = agg['revenue']/agg['entered']

convA = float(agg.loc[agg['variant']=='A','conv'])
convB = float(agg.loc[agg['variant']=='B','conv'])
rev_per_entered_A = float(agg.loc[agg['variant']=='A','rev_per_entered'])
rev_per_entered_B = float(agg.loc[agg['variant']=='B','rev_per_entered'])

# incremental conversions per 100k entered-checkout sessions:
entered_baseline = 100000
inc_conversions = (convB - convA) * entered_baseline
# incremental revenue estimate (using B revenue per entered or use A baseline rev_per_entered)
inc_revenue = inc_conversions * (rev_per_entered_B)  # conservative: use treatment revenue per entered

print(f"Conversion A: {convA:.4f}, Conversion B: {convB:.4f}")
print(f"Incremental conversions per {entered_baseline:,} entered sessions: {inc_conversions:,.0f}")
print(f"Estimated incremental revenue (using treatment rev_per_entered): ${inc_revenue:,.2f}")

# If you want per day estimate, provide average daily entered sessions:
df['date'] = df['first_ts'].dt.date
daily_entered = df[df['entered_checkout']==1].groupby('date')['entered_checkout'].sum().mean()
print(f"Average daily entered-checkout sessions (observed): {daily_entered:.0f}")
print(f"Estimated extra conversions per day: {(convB-convA)*daily_entered:.1f}")
print(f"Estimated extra revenue per day: {(convB-convA)*daily_entered*rev_per_entered_B:.2f}")


/tmp/ipython-input-2578858871.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  convA = float(agg.loc[agg['variant']=='A','conv'])
/tmp/ipython-input-2578858871.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  convB = float(agg.loc[agg['variant']=='B','conv'])
/tmp/ipython-input-2578858871.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  rev_per_entered_A = float(agg.loc[agg['variant']=='A','rev_per_entered'])
/tmp/ipython-input-2578858871.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  rev_per_entered_B = float(agg.loc[agg['variant']=='B','rev_per_entered'])


Conversion A: 0.4445, Conversion B: 0.5352
Incremental conversions per 100,000 entered sessions: 9,066
Estimated incremental revenue (using treatment rev_per_entered): $1,429,971.89
Average daily entered-checkout sessions (observed): 58111
Estimated extra conversions per day: 5268.5
Estimated extra revenue per day: 830977.64


In [11]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Prepare data for regression: filter for sessions where 'entered_checkout' is 1
sample_df = df[df['entered_checkout'] == 1].copy()

# Create 'treatment' variable (0 for A, 1 for B)
sample_df['treatment'] = (sample_df['variant'] == 'B').astype(int)

# Define the model formula (predict completed_sim based on treatment)
formula = 'completed_sim ~ treatment'

# Fit a logistic regression model
# Logit is appropriate for binary outcomes (0 or 1 for completed_sim)
model = smf.logit(formula=formula, data=sample_df).fit(disp=False) # disp=False to suppress optimization output

# 1) Print treatment coefficient (you already saw this)
try:
    coef_row = model.summary2().tables[1].loc['treatment']
    print("Regression coefficient row for 'treatment':\n")
    print(coef_row)
except Exception as e:
    print("Could not print coef row from model.summary2():", e)

# 2) Extract average marginal effects (AMCE) robustly using summary_frame()
try:
    margeff = model.get_margeff(at='overall', method='dydx')

    me_df = margeff.summary_frame()
    print("\nMarginal effects summary frame (first 10 rows):")
    display(me_df.head(10))

    if 'treatment' in me_df.index:
        tr_me = me_df.loc['treatment']
        print("\nAverage marginal effect for 'treatment':")
        print(tr_me)
    else:

        print("\n'me_df' index does not contain 'treatment'; available index values:")
        print(me_df.index.tolist())
except Exception as e:
    print("Could not compute/get margeff.summary_frame():", e)

# 3) Explicit ATE on probability scale (model predictions) — transparent check

df0 = sample_df.copy()
df1 = sample_df.copy()
df0['treatment'] = 0
df1['treatment'] = 1


try:
    pred0 = model.predict(df0)
    pred1 = model.predict(df1)

    mean_p0 = pred0.mean()
    mean_p1 = pred1.mean()
    ate = mean_p1 - mean_p0
    print("\nExplicit ATE (average predicted probability difference, treatment=1 minus treatment=0):")
    print(f"Mean p (treatment=0): {mean_p0:.6f}")
    print(f"Mean p (treatment=1): {mean_p1:.6f}")
    print(f"ATE (p1 - p0): {ate:.6f}  => {ate*100:.3f} percentage points")
except Exception as e:
    print("Prediction-based ATE computation failed:", e)

# 4) Quick sanity: print counts by treatment in sample used
print("\nSample size used for regression:", len(sample_df))
print("Treatment distribution in sample:")
print(sample_df['treatment'].value_counts())

Regression coefficient row for 'treatment':

Coef.         0.374071
Std.Err.      0.003085
z           121.245338
P>|z|         0.000000
[0.025        0.368024
0.975]        0.380118
Name: treatment, dtype: float64

Marginal effects summary frame (first 10 rows):


,dy/dx,Std. Err.,z,Pr(>|z|),Conf. Int. Low,Cont. Int. Hi.
treatment,0.090242,0.000732,123.278961,0.0,0.088808,0.091677



Average marginal effect for 'treatment':
dy/dx               0.090242
Std. Err.           0.000732
z                 123.278961
Pr(>|z|)            0.000000
Conf. Int. Low      0.088808
Cont. Int. Hi.      0.091677
Name: treatment, dtype: float64

Explicit ATE (average predicted probability difference, treatment=1 minus treatment=0):
Mean p (treatment=0): 0.372888
Mean p (treatment=1): 0.463620
ATE (p1 - p0): 0.090732  => 9.073 percentage points

Sample size used for regression: 1743344
Treatment distribution in sample:
treatment
0    873496
1    869848
Name: count, dtype: int64


In [8]:

import plotly.express as px
import plotly.graph_objects as go
from statsmodels.stats.proportion import proportions_ztest

# 1) Conversion bar with CI (reuse earlier logic)
entered_by_var = df.groupby('variant')['entered_checkout'].sum()
completed_by_var = df.groupby('variant')['completed_sim'].sum()
convA = completed_by_var['A'] / entered_by_var['A']
convB = completed_by_var['B'] / entered_by_var['B']

variants = ['A','B']
convs = [convA*100, convB*100]
entries = [int(entered_by_var['A']), int(entered_by_var['B'])]
comps = [int(completed_by_var['A']), int(completed_by_var['B'])]

fig_conv = go.Figure()
fig_conv.add_trace(go.Bar(x=variants, y=convs, text=[f"{c:.2f}%" for c in convs], textposition='auto'))
for i,(comp,n) in enumerate(zip(comps,entries)):
    p = comp/n
    se = np.sqrt(p*(1-p)/n)
    low = max(0,(p-1.96*se)*100)
    high = min(100,(p+1.96*se)*100)
    fig_conv.add_shape(type='rect', x0=i-0.15, x1=i+0.15, y0=low, y1=high, fillcolor='rgba(0,0,0,0.06)', line_width=0)
fig_conv.update_layout(title="Conversion Rate by Variant (with 95% CI)", yaxis_title="Conversion %")

# 2) Funnel counts
funnel = []
for v in variants:
    sub = df[df['variant']==v]
    funnel.append({'variant':v,'view':sub['views'].sum(),'cart':sub['carts'].sum(),'purchase':sub['purchases'].sum()})
funnel_df = pd.DataFrame(funnel)
funnel_long = funnel_df.melt(id_vars='variant', value_vars=['view','cart','purchase'], var_name='stage', value_name='count')
fig_funnel = px.bar(funnel_long, x='stage', y='count', color='variant', barmode='group', title='Funnel counts by Variant')

# 3) Cumulative conversion over time
tmp = df[df['entered_checkout']==1].copy()
tmp['date'] = tmp['first_ts'].dt.floor('D')
agg = tmp.groupby(['date','variant']).agg(entered=('entered_checkout','sum'), completed=('completed_sim','sum')).reset_index()
agg = agg.sort_values('date')
agg['cum_entered'] = agg.groupby('variant')['entered'].cumsum()
agg['cum_completed'] = agg.groupby('variant')['completed'].cumsum()
agg['cum_conv'] = agg['cum_completed']/agg['cum_entered']
fig_cum = px.line(agg, x='date', y='cum_conv', color='variant', title='Cumulative conversion over time')
fig_cum.update_yaxes(tickformat='.2%')

# 4) Category uplift bar (top 20)
cat = df.groupby(['parent_category','variant']).agg(entered=('entered_checkout','sum'), completed=('completed_sim','sum')).reset_index()
rows=[]
for c in sorted(cat['parent_category'].dropna().unique()):
    sub = cat[cat['parent_category']==c]
    a_enter = int(sub[sub['variant']=='A']['entered'].sum()) if not sub.empty else 0
    a_comp = int(sub[sub['variant']=='A']['completed'].sum()) if not sub.empty else 0
    b_enter = int(sub[sub['variant']=='B']['entered'].sum()) if not sub.empty else 0
    b_comp = int(sub[sub['variant']=='B']['completed'].sum()) if not sub.empty else 0
    convA = a_comp/a_enter if a_enter>0 else np.nan
    convB = b_comp/b_enter if b_enter>0 else np.nan
    rows.append({'category':c,'convA':convA,'convB':convB,'uplift':convB-convA,'A_enter':a_enter,'B_enter':b_enter})
cat_df = pd.DataFrame(rows).sort_values('uplift', ascending=False)
fig_cat = px.bar(cat_df.head(20), x='category', y='uplift', title='Top 20 Category Uplifts (B - A)')
fig_cat.update_xaxes(tickangle=45)

# 5) Price & duration uplift
price = df.groupby(['price_band','variant']).agg(entered=('entered_checkout','sum'), completed=('completed_sim','sum')).reset_index()
price_rows=[]
for b in price['price_band'].cat.categories:
    a_enter = int(price[(price['price_band']==b)&(price['variant']=='A')]['entered'].sum())
    a_comp = int(price[(price['price_band']==b)&(price['variant']=='A')]['completed'].sum())
    b_enter = int(price[(price['price_band']==b)&(price['variant']=='B')]['entered'].sum())
    b_comp = int(price[(price['price_band']==b)&(price['variant']=='B')]['completed'].sum())
    convA = a_comp/a_enter if a_enter>0 else np.nan
    convB = b_comp/b_enter if b_enter>0 else np.nan
    price_rows.append({'price_band':b,'uplift':convB-convA})
price_df = pd.DataFrame(price_rows)
fig_price = px.bar(price_df, x='price_band', y='uplift', title='Uplift by Price Band')

dur = df.groupby(['duration_bucket','variant']).agg(entered=('entered_checkout','sum'), completed=('completed_sim','sum')).reset_index()
dur_rows=[]
for b in dur['duration_bucket'].cat.categories:
    a_enter = int(dur[(dur['duration_bucket']==b)&(dur['variant']=='A')]['entered'].sum())
    a_comp = int(dur[(dur['duration_bucket']==b)&(dur['variant']=='A')]['completed'].sum())
    b_enter = int(dur[(dur['duration_bucket']==b)&(dur['variant']=='B')]['entered'].sum())
    b_comp = int(dur[(dur['duration_bucket']==b)&(dur['variant']=='B')]['completed'].sum())
    convA = a_comp/a_enter if a_enter>0 else np.nan
    convB = b_comp/b_enter if b_enter>0 else np.nan
    dur_rows.append({'duration_bucket':b,'uplift':convB-convA})
dur_df = pd.DataFrame(dur_rows)
fig_dur = px.bar(dur_df, x='duration_bucket', y='uplift', title='Uplift by Duration')

# Show plots
fig_conv.show()
fig_funnel.show()
fig_cum.show()
fig_cat.show()
fig_price.show()
fig_dur.show()


/tmp/ipython-input-3284213400.py:63: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipython-input-3284213400.py:76: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [14]:
!pip install --quiet reportlab

from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import os

out_pdf = "/content/drive/MyDrive/ab_report_summary.pdf"
c = canvas.Canvas(out_pdf, pagesize=A4)
width, height = A4

# Title page
c.setFont("Helvetica-Bold", 16)
c.drawString(40, height-80, "A/B Test — Checkout Flow: Executive Summary")
c.setFont("Helvetica", 11)
c.drawString(40, height-110, f"Rows analyzed: {len(df):,}  |  Unique users: {df['user_id'].nunique():,}")
c.drawString(40, height-130, f"Conversion A: {convA:.4f} ({convA*100:.2f}%), Conversion B: {convB:.4f} ({convB*100:.2f}%)")
c.drawString(40, height-150, f"Absolute uplift (pp): {(convB-convA)*100:.2f} pp")

# Add images (ensure they exist)
imgs = [
    ("conversion_bar.png", 40, height-420, 500, 300),
    ("funnel_counts.png", 320, height-420, 500, 300),
    ("category_uplift.png", 40, height-760, 520, 300),
    ("cumulative_conversion.png", 320, height-760, 220, 300)
]
img_dir = "/content/drive/MyDrive/ab_figures"
ypos = height-200
for fname, x, y, w, h in imgs:
    p = os.path.join(img_dir, fname)
    if os.path.exists(p):
        try:
            img = ImageReader(p)
            c.drawImage(img, x, y-10, width=w, height=h)
        except Exception as e:
            print("Could not add image", p, "error:", e)
c.showPage()
c.save()
print("Saved PDF report to", out_pdf)

Saved PDF report to /content/drive/MyDrive/ab_report_summary.pdf
